In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob

In [24]:
#getting a list of files to be read
ex_table_files = glob.glob('*_extracted_table.txt')

df_list = []

for file in ex_table_files:
    df = pd.read_csv(file, delim_whitespace=True
                     , skiprows=1) 
    df = df[['p1','p2','rg','se','z','p']]
    df_list.append(df)

In [25]:
df_list[8].head()

,p1,p2,rg,se,z,p
0,munged/GOUT.sumstats.gz,munged/HOARDING.sumstats.gz,0.0428,0.1574,0.2717,0.7859
1,munged/GOUT.sumstats.gz,munged/IBS.sumstats.gz,-0.0137,0.0515,-0.2664,0.7899
2,munged/GOUT.sumstats.gz,munged/LUPUS.sumstats.gz,-0.1223,0.0838,-1.4606,0.1441
3,munged/GOUT.sumstats.gz,munged/MDD.sumstats.gz,0.0072,0.0118,0.6100,0.5418
4,munged/GOUT.sumstats.gz,munged/MIGRAINE.sumstats.gz,0.0344,0.0514,0.6698,0.5030
